The simple dependent click model is based on the following principles:
* the first result is always examined
* a result can only be examined if the previous result was examined
* stop if a clicked result is satisfactory (with probability 1-lambda_r)
* continue if result is not satisfactory

In [5]:
from random import random
from collections import Counter


class SdcmClickModel(object):
    """ Implements a Simple Dependent Click Model (SDCM). Generates clicks starting
        from the first rank, and stochastically decides if a clicked result is 
        satisfactory, using probabilities trained on a click log. If the result was 
        not satisfactory, the next result is examined, and then clicked with a 
        probability dependent on the corresponding relevance label. The probabilities
        for each of the relevance labels are taken from the book "Click Models for 
        Web Search".
    
    Args:
        click_log_path (str): Location of the click log.
        attr_model (str): Model used for attractiveness parameters
                            (perfect/navigational/informative)
    """

    """Click model attractiveness parameters"""
    attr_perf = {'HR':1.0, 'R':0.5, 'N':0.0}
    attr_nav = {'HR':0.95, 'R':0.5, 'N':0.05}
    attr_inf = {'HR':0.9, 'R':0.7, 'N':0.4} 
    
    def __init__(self, click_log_path='YandexRelPredChallenge.txt', attr_model='nav'):
        self.click_log_path = click_log_path
        if attr_model == 'nav':
            self.attr_model = SdcmClickModel.attr_nav
        elif attr_model == 'perf':
            self.attr_model = SdcmClickModel.attr_perf            
        elif attr_model == 'inf':
            self.attr_model = SdcmClickModel.attr_inf
        else:
            raise ValueError('The attractiveness model "{}" is not available.' \
                            ' Choose from "nav", "perf" or "inf".'.format(attr_model))
        self.params = self.learnParams()
            
    def learnParams(self):
        """ Learns parameters for simple dependent click model.

        Returns:
            dict: Learnt click model parameters.

        """

        with open(self.click_log_path,'r') as f:
            # global rank click counter
            rank_counter = Counter()
            # counter for clicks on consecutive ranks
            pair_counter = Counter()
            for line in f:
                line = line.split()
                time_passed = line[1]
                if time_passed == '0':
                    prev_clicked_rank = -100
                if line[2] == 'Q':
                    results = line[5:]
                else:
                    url = line[3]
                    try:
                        clicked_rank = results.index(url)
                    except: 
                        pass
                    rank_counter[clicked_rank] += 1
                    if clicked_rank == prev_clicked_rank + 1:
                        pair_counter[prev_clicked_rank] += 1
                    prev_clicked_rank = clicked_rank
        lambdas = {r+1:pair_counter[r]/rank_counter[r] for r in pair_counter.keys()}
        return lambdas

    
    def attrProbs(self, ranking):
        """ Assigns attractiveness probabilities for list of relevance labels.

        Args:
            ranking (list): Ranked list of relevance labels.

        Returns:
            list: List of attractiveness probabilities corresponding with ranking.

        """
        probs = [self.attr_model[label] for i, label in enumerate(ranking)]
        return probs

    def assignClicks(self, ranking):
        """ Assigns clicks based on attractiveness and satisfactoriness probabilities.

        Args:
            ranking (list): Ranked list of relevance labels.

        Returns:
            list: Assigned clicks.

        """
        probs = self.attrProbs(ranking)
        
        clicks = []  
        for i in range(len(probs)):
            if i == 0:
                # for first result, click probability equals attractiveness
                p_click = probs[i]
            else:
                prev_click = clicks[i-1]
                if prev_click == 1:
                    p_satisf = 1 - self.params[i]
                    # stochastically decide satisfactoriness    
                    outcome = random()
                    if outcome < p_satisf:
                        # when the clicked result was satisfactory, stop
                        clicks += ([0] * (len(probs) - len(clicks)))
    #                     print('Result {} was satisfactory!'.format(i))
                        break
                    else:
                        p_click = probs[i]
                else:
                    p_click = probs[i]
            # stochastically decide click 
            outcome = random()
            if outcome < p_click:
                clicks.append(1)
            else: clicks.append(0)
        return clicks

In [6]:
model = SdcmClickModel()

In [20]:
ranking = ['HR', 'R', 'R', 'N', 'HR', 'N', 'R']
model.assignClicks(ranking)

[1, 1, 0, 0, 1, 0, 0]